In [4]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

# the following line is not required if BatchFlow is installed as a python package.
sys.path.append('../../../batchflow')
from batchflow import Dataset, V, F, B, action, Batch
from batchflow.models.torch import TorchModel
from batchflow.models.metrics import ClassificationMetrics
from batchflow.opensets import MNIST
from batchflow import best_practice
from batchflow.models.torch.layers import Dense

In [5]:
BATCH_SIZE = 64
dataset = MNIST()

Downloaded t10k-labels-idx1-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-labels-idx1-ubyte.gz
Downloaded train-labels-idx1-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-labels-idx1-ubyte.gz
Downloaded t10k-images-idx3-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-images-idx3-ubyte.gz
Downloaded train-images-idx3-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-images-idx3-ubyte.gz


In [6]:
class LinearModel(TorchModel):
    
    @classmethod
    def default_config(cls):
        config = TorchModel.default_config()
        config['body'].update(dict(num_classes=2, layout='f'))
        return config

    def build_config(self, names=['images', 'labels']):
        config = super().build_config(names)
        config['body/num_classes'] = self.num_classes('targets')
        return config

    @classmethod
    def body(cls, **kwargs):
        kwargs = cls.get_defaults('body', kwargs)
        x = Dense(**kwargs)
        return x

In [10]:
linear_model_config = {
    'loss': 'ce',
    'decay': 'exp',
    'optimizer': 'Adam',
    'inputs': dict(images={'shape': (128, 128, 3)},
                   labels={'classes': 10}),
    'initial_block/inputs': 'images'
}

pipeline = (dataset.p
    .init_variable('loss_history', init_on_each_run=list)
    .init_model('dynamic', LinearModel, 'my_model', linear_model_config)
    .train_model('my_model', B('images'), B('labels'),
                 fetches='loss', save_to=V('loss_history'), mode='a')
    .run(BATCH_SIZE, shuffle=True, n_epochs=5))

AttributeError: 'NoneType' object has no attribute 'gen_batch'